# Домашнее задание 3

Выполнил Козин Роман

# Часть 1. Классификатор 128×128

## Датасет

Для ускорения процесса обучения используем не весь датасет ImageNet, возьмём часть классов. Пусть будет 40 классов. Выберем их случайным образом

In [1]:
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import numpy as np
import random
import torch

In [2]:
random.seed(52)
torch.manual_seed(52)
np.random.seed(52)

In [3]:
CLASS_NUM = 40
classes_path = './data/tiny-imagenet-200/tiny-imagenet-200/wnids.txt'
with open(classes_path) as f:
    ids = f.read().split('\n')
selected_classes = random.sample(ids[:-1], CLASS_NUM)
print(selected_classes)

['n01882714', 'n02279972', 'n02699494', 'n02123394', 'n03026506', 'n01910747', 'n04532106', 'n07920052', 'n02058221', 'n07695742', 'n02268443', 'n07753592', 'n03160309', 'n04356056', 'n01784675', 'n02124075', 'n07871810', 'n04417672', 'n02927161', 'n03837869', 'n04540053', 'n09193705', 'n02364673', 'n02909870', 'n01774750', 'n12267677', 'n03404251', 'n01698640', 'n02988304', 'n01770393', 'n03930313', 'n02977058', 'n02963159', 'n01644900', 'n02415577', 'n03992509', 'n07875152', 'n02814533', 'n02395406', 'n02814860']


На всякий случай зафиксируем

In [4]:
selected_classes = ['n01882714', 'n02279972', 'n02699494', 'n02123394', 'n03026506', 'n01910747', 'n04532106', 'n07920052', 
                    'n02058221', 'n07695742', 'n02268443', 'n07753592', 'n03160309', 'n04356056', 'n01784675', 'n02124075', 
                    'n07871810', 'n04417672', 'n02927161', 'n03837869', 'n04540053', 'n09193705', 'n02364673', 'n02909870', 
                    'n01774750', 'n12267677', 'n03404251', 'n01698640', 'n02988304', 'n01770393', 'n03930313', 'n02977058', 
                    'n02963159', 'n01644900', 'n02415577', 'n03992509', 'n07875152', 'n02814533', 'n02395406', 'n02814860']

Возьмём ту же реализацию датасета из прошлого ДЗ, останется только добавить resize в изображения. Делать дополнительной стратификации для разделения датасета на обучение и валидацию не нужно, в ImageNet датасет уже разбит на нужные выборки.

In [7]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn as nn
from tqdm import tqdm
from PIL import Image
import torch
import os

class SmallDataset(Dataset):
    def __init__(self, root: str, classes: list[str], ds_type: str, device = torch.device("cpu"), transform = None) -> None:
        super().__init__()
        self.ds_type = ds_type
        self.root = root
        self.classes = classes
        if ds_type == 'val':
            self.prepareValData()
        self.device = device
        self.transform = transform
        self.label_to_idx = {label : idx for idx, label in enumerate(classes)}
        
    def prepareValData(self) -> None:
        self.valData = []
        labelsSet = set(self.classes)
        with open(os.path.join(self.root, self.ds_type, "val_annotations.txt")) as f:
            rows = f.readlines()
            f.close()    
        for row in rows:
            data = self._parseLine(row)
            label = data[1]
            if label in labelsSet:
                self.valData.append(tuple(data))
    
    def _parseLine(self, row: str):
        els = row.split("\t")
        data = [els[k].strip("\n") for k in range(len(els))]
        if self.ds_type == 'val':
            start_idx = 2
        else:
            start_idx = 1
        for i in range(start_idx, len(data)):
            data[i] = int(data[i])
        return data
    
    def __getitem__(self, index: int):
        if self.ds_type == 'val':
            data = self.valData[index]
            img_path = os.path.join(self.root, self.ds_type, "images", data[0])
            label = data[1]
            bbox = tuple(data[2:])
        else:
            label_idx = index // 500
            img_idx = index % 500
            label = self.classes[label_idx]
            label_root = os.path.join(self.root, self.ds_type, label)
            img_path = os.path.join(label_root, "images", f'{label}_{img_idx}.JPEG')
            with open(os.path.join(label_root, f"{label}_boxes.txt")) as f:
                rows = f.readlines()
                f.close()
            row = rows[img_idx]
            data = self._parseLine(row)
            bbox = tuple(data[1:])
        
        bbox = torch.tensor(bbox).to(self.device)
        img = Image.open(img_path).convert('RGB')
            
        if self.transform:
            img = self.transform(img)
        return (img.to(self.device), torch.tensor(self.label_to_idx[label]).to(self.device), bbox)
        
    def __len__(self) -> int:
        #структура исходного датасета такова, что на каждый класс приходится 500 изображений
        if self.ds_type == 'train':
            return len(self.classes) * 500
        else:
            return len(self.valData)

Увеличим размер изображений до 128x128 при помощи ```transforms.Resize()```, используем стандартную билинейную интерполяцию. Также в тренировку добавим несколько аугментаций для более стабильного обучения. Добавим аугментации, предложенные в задании: RandomResizedCrop, Flip, Rotation.

В RandomResizedCrop выставим адекватные размеры обрезки, чтобы не оставлять в датасете слишком маленькие участки исходного изображения.

В RandomRotation используем повороты до 30 градусов, чтобы тоже слишком сильно не искажать изображения.

In [10]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128, 128)),
    transforms.RandomResizedCrop((128, 128), (0.6, 1)),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], #найденные в интернете параметры для imagenet
                        std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128, 128)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], #найденные в интернете параметры для imagenet
                        std=[0.229, 0.224, 0.225])
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_root = "./data/tiny-imagenet-200/tiny-imagenet-200"
train_data = SmallDataset(root=data_root, classes=selected_classes,
                          ds_type='train', transform=train_transforms, device=device)
val_data = SmallDataset(root=data_root, classes=selected_classes,
                        ds_type='val', transform=transforms_list, device=device)
train_data_loader = DataLoader(train_data,batch_size=16, shuffle=True)
val_data_loader = DataLoader(val_data, batch_size=16, shuffle=True)

In [14]:
img, label, bbox = train_data[0]

print(img.shape)

torch.Size([3, 128, 128])
